This function will take in tickers array and make a MultiIndexed pandas dataframe with date as datetime and arranges from latest to earliest for last 'filter' years, woth default value being 10 years

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
def load_data(tickers,filter=10):
    dataframes=[]
    for ticker in tickers:
        df=pd.read_csv(f'{ticker}.us.txt')
        df["TICKER"]=ticker.upper()
        dataframes.append(df)
    df_new=pd.concat(dataframes)  
    df_new['Date']=pd.to_datetime(df_new['Date'])
    df_new.set_index(['TICKER','Date'], inplace=True)
    df_new.sort_index(ascending=[True,False],inplace=True)
    latest_date=df_new.index.get_level_values('Date').max()
    cutoff_date=latest_date - pd.Timedelta(days=365*filter)
    df_new=df_new[df_new.index.get_level_values('Date') >= cutoff_date]
    return df_new



Now we check if there are any null values in any column, if there aren't, filling is not required. Function parameter is dataframe

In [90]:
def check_null(df):
    nulls = df.isnull().sum()
    nulls = nulls[nulls > 0]
    if nulls.empty:
        print("No null values found.")
    else:
        print("Null values found in the following columns:")
        print(nulls)
        # plt.figure(figsize=(10, 6))
        # nulls.plot(kind='bar')
        # plt.title('Null Values in DataFrame')
        # plt.xlabel('Columns')
        # plt.ylabel('Number of Nulls')
        # plt.show()
    return None

This is a fxn to remove all zero columns from dataframe

In [91]:
def zero_columns(df):
    zero_columns=(df==0).all()
    zero_columns = zero_columns[zero_columns]
    print(zero_columns.index.tolist())
    return zero_columns.index.tolist()

def rem_col(name,df):
    df.drop(columns=[name], inplace=True)
    return df

Now we add the desired columns. Note that dates should be in earliest to latest otherwise stock would look into future

In [92]:
def add_dailyReturn (df):
    df.sort_index(ascending=[True,True],inplace=True)
    df['Daily_Return']= df.groupby(level='TICKER')['Close'].pct_change()
    df.sort_index(ascending=[True,False],inplace=True)
    return df

def add_7day_avg(df):
    df.sort_index(ascending=[True,True],inplace=True)
    df['7_day_avg'] = df.groupby(level='TICKER')['Close'].transform(lambda x: x.rolling(window=7).mean())
    df.sort_index(ascending=[True,False],inplace=True)
    return df

def add_30day_avg(df):
    df.sort_index(ascending=[True,True],inplace=True)
    df['30_day_avg'] = df.groupby(level='TICKER')['Close'].transform(lambda x: x.rolling(window=30).mean())
    df.sort_index(ascending=[True,False],inplace=True)
    return df

def add_30dayRollingVolatitility(df):
    df.sort_index(ascending=[True,True],inplace=True)
    df['30_day_rolling_volatility'] = df.groupby(level='TICKER')['Close'].transform(lambda x: x.pct_change().rolling(window=30).std())
    df.sort_index(ascending=[True,False],inplace=True)
    return df



For exploratory analysis, we define some functions for avg_returns which is mean of Daily_Return column (should be added before using this fxn). Also note that this function output is fractional avg return and volatility, for percentage, multiply by 100.

In [93]:
def highestAvgReturn(df):
    avg_returns=df.groupby(level='TICKER')['Daily_Return'].mean()
    highest_avg_return_stock = avg_returns.idxmax()
    highest_avg_return_value = avg_returns.max()
    print(avg_returns)
    print(f"Highest average return stock: {highest_avg_return_stock} with avg return of {highest_avg_return_value:.4f} or {highest_avg_return_value*100:.2f}%")

def highestVolatility(df):
    df_reset=df.reset_index()
    df_reset["YearMonth"] = df_reset["Date"].dt.to_period("M")
    monthly_vol=df_reset.groupby(['TICKER', 'YearMonth'])['30_day_rolling_volatility'].mean()
    
    most_volatile = monthly_vol.idxmax()
    volatility_value = monthly_vol.max()
    print(f"Most volatile stock: {most_volatile[0]} in {most_volatile[1]} with volatility of {volatility_value:.4f} or {volatility_value*100:.2f}%") 
    # return monthly_vol








Now we import the dataset and apply our functions

In [94]:
tickers=['aapl','brk-b','googl','msft','nvda','tsla']
df=load_data(tickers)
print(df.head())

                     Open    High     Low   Close    Volume  OpenInt
TICKER Date                                                         
AAPL   2017-11-10  175.11  175.38  174.27  174.67  25130494        0
       2017-11-09  174.48  175.46  172.52  175.25  29533086        0
       2017-11-08  174.03  175.61  173.71  175.61  24451166        0
       2017-11-07  173.29  174.51  173.29  174.18  24424877        0
       2017-11-06  171.75  174.36  171.10  173.63  34901241        0


In [95]:
check_null(df)

No null values found.


In [96]:
zero_columns(df)

['OpenInt']


['OpenInt']

Only 'OpenInt' column is zero columns, we can remove it safely

In [97]:
rem_col('OpenInt',df)
print(df.head())

                     Open    High     Low   Close    Volume
TICKER Date                                                
AAPL   2017-11-10  175.11  175.38  174.27  174.67  25130494
       2017-11-09  174.48  175.46  172.52  175.25  29533086
       2017-11-08  174.03  175.61  173.71  175.61  24451166
       2017-11-07  173.29  174.51  173.29  174.18  24424877
       2017-11-06  171.75  174.36  171.10  173.63  34901241


now we add the columns of averages and volatility

In [98]:
add_dailyReturn(df)
add_7day_avg(df)
add_30day_avg(df)
add_30dayRollingVolatitility(df)

Open    High     Low   Close    Volume  Daily_Return  \
TICKER Date                                                                 
AAPL   2017-11-10  175.11  175.38  174.27  174.67  25130494     -0.003310   
       2017-11-09  174.48  175.46  172.52  175.25  29533086     -0.002050   
       2017-11-08  174.03  175.61  173.71  175.61  24451166      0.008210   
       2017-11-07  173.29  174.51  173.29  174.18  24424877      0.003168   
       2017-11-06  171.75  174.36  171.10  173.63  34901241      0.010182   
...                   ...     ...     ...     ...       ...           ...   
TSLA   2010-07-02   23.00   23.10   18.71   19.20   5141807     -0.125683   
       2010-07-01   25.00   25.92   20.27   21.96   8229863     -0.078473   
       2010-06-30   25.79   30.42   23.30   23.83  17194394     -0.002512   
       2010-06-29   19.00   25.00   17.54   23.89  18783276      0.405294   
       2010-06-28   17.00   17.00   17.00   17.00         0           NaN   

                    7_day_avg  30_day_avg  30_day_rolling_volatility  
TICKER Date                                                           
AAPL   2017-11-10  173.247143  161.285333                   0.011503  
       2017-11-09  172.050000  160.582000                   0.011412  
       2017-11-08  171.075714  159.831333                   0.011520  
       2017-11-07  169.720000  159.100333                   0.011510  
       2017-11-06  168.047143  158.380667                   0.011738  
...                       ...         ...                        ...  
TSLA   2010-07-02         NaN         NaN                        NaN  
       2010-07-01         NaN         NaN                        NaN  
       2010-06-30         NaN         NaN                        NaN  
       2010-06-29         NaN         NaN                        NaN  
       2010-06-28         NaN         NaN                        NaN  

[14447 rows x 9 columns]

Highest average return

In [99]:
highestAvgReturn(df)

TICKER
AAPL     0.001022
BRK-B    0.000388
GOOGL    0.000627
MSFT     0.000577
NVDA     0.001239
TSLA     0.002101
Name: Daily_Return, dtype: float64
Highest average return stock: TSLA with avg return of 0.0021 or 0.21%


In [100]:
highestVolatility(df)

Most volatile stock: NVDA in 2008-11 with volatility of 0.0762 or 7.62%


In [105]:
last_dates=df.groupby(level='TICKER').apply(lambda x: x.index.get_level_values('Date').min())
print("Last dates for each ticker:")
print(last_dates)

Last dates for each ticker:
TICKER
AAPL    2007-11-13
BRK-B   2007-11-13
GOOGL   2007-11-13
MSFT    2007-11-13
NVDA    2007-11-13
TSLA    2010-06-28
dtype: datetime64[ns]
